In [13]:
import time

import pymysql
from sshtunnel import SSHTunnelForwarder


def get_ec2_config():
    return {
        "host_name": "3.114.154.242",
        "ec2_port": 22,
        "ssh_username": "ec2-user",
        "ssh_pkey": "~/.ssh/vook-rails-ssh-key.pem",
        "rds_end_point": "vook-rails-db.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com",
        "rds_port": 3306,
    }


def get_rds_config(port):
    return {
        "user": "root",
        "password": "rds-vook",
        "port": port,
        "host": "localhost",
        "database": "vook_web_v3_production",
        "charset": "utf8mb4",
        "cursorclass": pymysql.cursors.DictCursor,
    }


config_ec2 = get_ec2_config()

# SSHトンネルの設定
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None

    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port), connect_timeout=10
        )
        cursor = conn.cursor()
        # SQLクエリの実行
        query = "SELECT * FROM knowledges limit 5"
        cursor.execute(query)
        # 結果の取得と表示
        # for (
        #     knowledge_id,
        #     name,
        #     age,
        #     brand_id,
        #     item_id,
        #     line_id,
        #     created_at,
        #     updated_at,
        #     user_id,
        # ) in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
        #     print(f"{knowledge_id}")
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            print(row)

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 53769
{'id': 5, 'name': '66前期', 'age': 1974, 'brand_id': 1, 'item_id': 1, 'line_id': 1, 'created_at': datetime.datetime(2023, 7, 9, 2, 18, 48, 15596), 'updated_at': datetime.datetime(2023, 10, 22, 6, 38, 57, 230064), 'user_id': 3}
{'id': 8, 'name': '1st', 'age': 1960, 'brand_id': 7, 'item_id': 2, 'line_id': 17, 'created_at': datetime.datetime(2023, 7, 9, 3, 0, 6, 265820), 'updated_at': datetime.datetime(2023, 10, 19, 23, 19, 34, 358783), 'user_id': 3}
{'id': 10, 'name': 'BIG E', 'age': 1970, 'brand_id': 1, 'item_id': 1, 'line_id': 1, 'created_at': datetime.datetime(2023, 7, 11, 0, 31, 57, 807324), 'updated_at': datetime.datetime(2023, 11, 6, 23, 53, 49, 259033), 'user_id': 3}
{'id': 11, 'name': '片面タブ', 'age': 1947, 'brand_id': 1, 'item_id': 1, 'line_id': 2, 'created_at': datetime.datetime(2023, 7, 11, 0, 36, 8, 620636), 'updated_at': datetime.datetime(2023, 10, 24, 10, 52, 51, 183702), 'user_id': 3}
{'id': 12, 'name': '両面タブ', 'age': 1953, 'brand_id': 1, 'item_id': 1, '